In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#  Packages & Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import sklearn as skl
import warnings
import statsmodels.api as sm

from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict, ShuffleSplit, validation_curve, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler, LabelEncoder, scale, MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification

from xgboost import XGBClassifier 

from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter('ignore')


In [5]:
%load_ext autoreload
%aimport data_prep
%run data_prep.py
%aimport modeling
%run modeling.py
%autoreload 1

## Load prepared Data

In [120]:
#Load Datasets
data = pd.read_csv("./FE_Final/train_data_prepared.csv")
target = pd.read_csv("./original_datasets/training_set_labels.csv")
target=target[["status_group"]]
# data  = data.drop('subvillage',axis=1)


# Baseline Model (Random Forest)

In [9]:


X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.8)

# rf = RFC(n_estimators = 1000, random_state = 42)

# rf.fit(X_train, y_train)

# rf_score = rf.score(X_test,y_test)

# print('Accuracy:', rf_score)


# Multi-class logReg

In [20]:
clf = LogisticRegression(multi_class="ovr").fit(data, target)
clf_Score = clf.score(X_test,y_test)
print('Accuracy:', clf_Score)

Accuracy: 0.7319023569023569


In [14]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(X_train.drop(['gps_height','latitude'],axis=1), y_train)
mnb.score(X_test.drop(['gps_height','latitude'], axis=1),y_test)

0.4521043771043771

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.8)

# rf = RFC(n_estimators = 1000, random_state = 42)

# rf.fit(X_train, y_train)

# rf_score = rf.score(X_test,y_test)

# print('Accuracy:', rf_score)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import KFold

def grid_search(Xtrain,ytrain, k):
     cv = KFold(k)
    rfc= Pipeline((
    ('one_hot', one_hot()),
    ('rfc', RandomForestClassifier())
    ))

    knn = Pipeline((
    ('choose_geo', filter_d()),
    ('knn', KNeighborsClassifier())
    ))

    svc = Pipeline((
    ('svc', SVC()),
    ))

    lr = Pipeline((
    ('lr',LogisticRegression(multi_class="ovr")),
    ))

    rfc_parameters = {
    'rfc__n_estimators': [10, 20, 30],
    'rfc__criterion': ['gini', 'entropy', 'accuracy'],
    'rfc__max_features': [5, 10, 15],
    'rfc__max_depth': ['auto', 'log2', 'sqrt', None]
    }

    knn_parameters = {
    'knn__n_neighbors': [3, 7, 10],
    'knn__weights': ['uniform', 'distance']
    }

    svc_parameters = {
    'svc__C': [0.01, 0.1, 1.0],
    'svc__kernel': ['rbf', 'poly'],
    'svc__gamma': [0.01, 0.1, 1.0],

    }
    lr_parameters = {
    'mnb__alpha': [0.01, 0.1, 1.0]
    }

    pars = [parameters1, parameters2, parameters3, parameters4]
    pips = [pipeline1, pipeline2, pipeline3, pipeline4]
    
    d_pred = {}
    print "starting Gridsearch"
    for i in range(len(pars)):
        gs = GridSearchCV(pips[i], pars[i],cv =  ,  verbose=2, refit=False, n_jobs=-1)
        gs = gs.fit(X_train, y_train)
        d_pred[pips[i]] = gs.best_estimator_
        print "finished Gridsearch"
        print gs.best_score_

In [ ]:
y = target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [46]:

RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[42]}
rf = GridSearchCV(RFC(),
                                 param_grid= RF_parameters,
                                 cv=KFold(5))



rf.fit(X, y)
rf.cv_results_
feature_importances = pd.DataFrame(rf.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)                                                                 ascending=False)


In [47]:
feature_importances = pd.DataFrame(rf.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [53]:
rf.best_score_

0.8109259259259259

In [90]:
Random_f = rf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=32, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [105]:
m = RFC(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=32, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
m.fit(X_train[np.array(fo5)],y_train)
m.score(X_test[np.array(fo5)] , y_test)

0.8046464646464646

In [106]:
#rf.cv_results_

In [ ]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300],
              'random_state':[random_seed]
}

model = xgb.XGBClassifier()


to_int = {'functional':1,'non functional':2,'functional needs repair':3}
to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

y_Train = dataset['status_group'].loc[(dataset['is_test'].isin([0]) )]
y_Train   = y_Train.replace(to_int).copy()
dataset = dataset.drop('status_group',axis=1).copy()
   
    ### Label encode
dataset = label_encoder(dataset).copy()
    ### Divide test and train 
X_Train= dataset.loc[(dataset['is_test'].isin([0]) )].drop('is_test',axis=1)
test = dataset.loc[(dataset['is_test'].isin([1]) )].drop(['is_test'],axis=1)

    #### Set ID aside for test 
ID = test['id'] 

    ### If train_id set to Trues the the ID will be kept as feature

X_Train = X_Train.set_index('id')
test = test.set_index('id')
# import re

# X_train = X_train.rename(lambda x: re.sub("[\]\(\]\)\, ]","",x), axis='columns')
# X_test = X_test.rename(lambda x: re.sub("[\]\(\]\)\, ]","",x), axis='columns')

xg = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = KFold(5,random_state=random_seed),
                    n_jobs = -1,
                    scoring = 'accuracy',
                    verbose=True)
xg.fit(X_Train,y_Train)
print(xg.cv_results_)
print(xg.best_params_)
print(xg.best_score_ )

In [116]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

rfc = RFC(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=32, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

test_pred1 ,train_pred1=Stacking(model=rfc , train=X_train,fold=KFold(5),test=X_test,y=y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                            'criterion':['mse']}
# rf = GridSearchCV(RandomForestRegressor(),
#                                  param_grid= RF_parameters,
#                                  cv=tscv)

# test_pred2 ,train_pred2=Stacking(model=rf ,train=xwe_Train,fold=tscv,test=xwe_Test,y=ywe_cnt_train)

# train_pred2=pd.DataFrame(train_pred2)
# test_pred2=pd.DataFrame(test_pred2)

# param_grid = {'learning_rate': [0.01, 0.15  ,0.2], 
#           'max_depth': [8,12 , 16],
#           'min_child_weight': [2, 3,5,10],
#           'subsample': [0.5,  0.8],
#           'colsample_bytree': [0.5, 0.75],
#           'n_estimators': [18, 50 , 100],
# }

# model = xgb.XGBRegressor()

# xg1 = GridSearchCV(model,
#                     param_grid = param_grid,
#                     cv = tscv,
#                     n_jobs = -1,
#                     scoring = 'r2',
#                     verbose=True)

# test_pred3 ,train_pred3=Stacking(model=xg1 ,train=xwe_Train,fold=tscv,test=xwe_Test,y=ywe_cnt_train)

# train_pred3=pd.DataFrame(train_pred3)
# test_pred3=pd.DataFrame(test_pred3)


##### if doing time series fold check teh number of rows that where skipped


0.8069584736251403
0.8026936026936027
0.8002244668911336
0.8088664421997755
0.8004489337822671


In [127]:
df = pd.concat([train_pred1, train_pred2, train_pred3,xwe_Train.drop(np.arange(0,795)).reset_index(drop=True) ], axis=1)
df_test = pd.concat([test_pred1, test_pred2,test_pred3,xwe_Test.reset_index(drop=True)], axis=1)

## Or any model 

param_grid = {'learning_rate': [0.01, 0.15  ,0.2], 
          'max_depth': [8,12 , 16],
          'min_child_weight': [2, 3,5,10],
          'subsample': [0.5,  0.8],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [18, 50 , 100],
}

model = xgb.XGBRegressor()

xg1 = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'r2',
                    verbose=True)

test_pred3 ,train_pred3=Stacking(model=xg1 ,train=xwe_Train,fold=tscv,test=xwe_Test,y=ywe_cnt_train)

train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)

NameError: name 'train_pred2' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_parameters = {'n_neighbors':[1, 5,10 ],'weights':['uniform'], 'algorithm':['auto'],'leaf_size':[5 ,10],
                  'p':[1,2,3],'metric':['minkowski']}
knn = GridSearchCV(KNeighborsClassifier(),verbose =True,
                                 param_grid=knn_parameters,
                                 cv=KFold(5,random_state = 42),return_train_score=True)
y = target
X =  onehot_encode(data.drop(['subvillage','ward'],axis=1))
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

knn.fit(X, y)
knn.cv_results_
feature_importances_knn = pd.DataFrame(rf.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)  

# of features before: 36
# of features after: 199
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [128]:
 Xll = data[['longitude','latitude']]
yll = target

In [129]:
knn_parameters = {'n_neighbors':[1, 5,10,15, 30 ],'weights':['uniform'], 'algorithm':['auto'],'leaf_size':[5 ,10, 20,30],
                  'p':[1,2,3],'metric':['minkowski']}
knn = GridSearchCV(KNeighborsClassifier(),verbose =True,
                                 param_grid=knn_parameters,
                                 cv=KFold(5,random_state = 42),return_train_score=True)
X_train, X_test, y_train, y_test = train_test_split(
    Xll, yll, random_state=42)
knn.fit(X, y)
knn.cv_results_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 275.1min finished


{'mean_fit_time': array([1.92421985, 1.87660894, 1.90447292, 1.91896057, 1.90645943,
        1.89739447, 1.89121313, 1.89411416, 1.88442364, 1.88729324,
        1.91022983, 1.90859437, 1.90047684, 1.9036797 , 1.89826627,
        1.83545651, 1.82385678, 1.85717387, 1.85031886, 1.84076877,
        1.81983633, 1.81899977, 1.84776735, 1.81984277, 1.85450344,
        1.83168035, 1.84337835, 1.81737838, 1.82294574, 1.81219687,
        1.7988162 , 1.77091265, 1.78091316, 1.8247786 , 1.79786363,
        1.79305634, 1.80452762, 1.77373061, 1.79080768, 1.82343669,
        1.84795156, 1.82584338, 1.84696603, 1.84036465, 1.83411822,
        1.8       , 1.81963778, 1.79566813, 1.80957804, 1.81271644,
        1.83556819, 1.80542331, 1.79194312, 1.79942851, 1.8311636 ,
        1.82364721, 1.80169592, 1.83410749, 1.82667546, 1.82292466]),
 'std_fit_time': array([0.08195366, 0.02479019, 0.01727934, 0.0576197 , 0.01091378,
        0.017919  , 0.02058137, 0.00683706, 0.01512461, 0.03149393,
        0.023

In [131]:
knn.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=5, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=1,
           weights='uniform')

In [ ]:
def label_encoder(df):
    categorical = categorical_features(df)
    # Creating the label encoder object
    le =  LabelEncoder()
    
    # Iterating over the "object" variables to transform the categories into numbers 
    for col in categorical:
        df[col] = le.fit_transform(df[col].astype(str))
    return df


In [136]:
data = pd.read_csv("./FE_steps/3 train_data_NOIMPUTE_age_dayse_recorded_density.csv")

In [141]:
 scores , preds = test_score(data ,'NOIMPUTE_age_dayse_recorded_density')

NameError: name 'LabelEncoder' is not defined

In [158]:
dataset = pd.read_csv('./original_datasets/initial_joined.csv')

In [ ]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300],
              'random_state':[random_seed]
}

model = xgb.XGBClassifier()


to_int = {'functional':1,'non functional':2,'functional needs repair':3}
to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

y_Train = dataset['status_group'].loc[(dataset['is_test'].isin([0]) )]
y_Train   = y_Train.replace(to_int).copy()
dataset = dataset.drop('status_group',axis=1).copy()
   
    ### Label encode
dataset = label_encoder(dataset).copy()
    ### Divide test and train 
X_Train= dataset.loc[(dataset['is_test'].isin([0]) )].drop('is_test',axis=1)
test = dataset.loc[(dataset['is_test'].isin([1]) )].drop(['is_test'],axis=1)

    #### Set ID aside for test 
ID = test['id'] 

    ### If train_id set to Trues the the ID will be kept as feature

X_Train = X_Train.set_index('id')
test = test.set_index('id')
# import re

# X_train = X_train.rename(lambda x: re.sub("[\]\(\]\)\, ]","",x), axis='columns')
# X_test = X_test.rename(lambda x: re.sub("[\]\(\]\)\, ]","",x), axis='columns')

xg = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = KFold(5,random_state=random_seed),
                    n_jobs = -1,
                    scoring = 'accuracy',
                    verbose=True)
xg.fit(X_Train,y_Train)
print(xg.cv_results_)
print(xg.best_params_)
print(xg.best_score_ )

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 53.5min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 159.5min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 291.9min


In [ ]:
predict = xg.predict(test)

In [10]:
dataset = pd.read_csv('./original_datasets/initial_joined.csv')

In [11]:
models = [
        
    RFC(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=32, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=random_seed, verbose=0, warm_start=False)
,
        
  XGBClassifier(learning_rate= 0.01, 
          max_depth= 8,
          min_child_weight= 10,
          subsample= 0.75,
          colsample_bytree=  0.75,
          n_estimators = 300,
              random_state=random_seed,
                  n_jobs=-1, verbose = True)
]

In [12]:
to_int = {'functional':1,'non functional':2,'functional needs repair':3}
to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

y_Train = dataset['status_group'].loc[(dataset['is_test'].isin([0]) )]
y_Train   = y_Train.replace(to_int).copy()
dataset = dataset.drop('status_group',axis=1).copy()
   
    ### Label encode
dataset = label_encoder(dataset).copy()
    ### Divide test and train 
X_Train= dataset.loc[(dataset['is_test'].isin([0]) )].drop('is_test',axis=1)
test = dataset.loc[(dataset['is_test'].isin([1]) )].drop(['is_test'],axis=1)

    #### Set ID aside for test 
ID = test['id'] 

    ### If train_id set to Trues the the ID will be kept as feature

X_Train = X_Train.set_index('id')
test = test.set_index('id')

In [18]:
from vecstack import stacking
S_train, S_test = stacking(models,                     # list of models
                           X_Train, y_Train, test,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=accuracy_score,      # metric: callable
                           n_folds=5,                  # number of folds
                           stratified=False,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=random_seed,             # ensure reproducibility
                           verbose=2)                  # print all info

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.81161616]
    fold  1:  [0.80993266]
    fold  2:  [0.80690236]
    fold  3:  [0.81489899]
    fold  4:  [0.81666667]
    ----
    MEAN:     [0.81200337] + [0.00348106]
    FULL:     [0.81200337]

model  1:     [XGBClassifier]
    fold  0:  [0.77297980]
    fold  1:  [0.77020202]
    fold  2:  [0.77230640]
    fold  3:  [0.78585859]
    fold  4:  [0.77760943]
    ----
    MEAN:     [0.77579125] + [0.00558471]
    FULL:     [0.77579125]



In [33]:

NX_Train = pd.DataFrame(S_train)

scores= cross_val_score(XGBClassifier(), 
                NX_Train, y_Train,scoring='accuracy', cv=KFold(5,random_state= random_seed))

In [34]:
scores

array([0.81683502, 0.80808081, 0.81430976, 0.80934343, 0.81094276])